In [0]:
import numpy as np
from scipy.optimize import linprog

def nn2na(NN):
    # Get every location where exist an arc:
    idxs = np.argwhere(NN)
    # Preallocate NA matrix, dimension is (nodes, arcs)
    NA = np.zeros([NN.shape[0], idxs.shape[0]]).astype(int)
    #C = np.zeros(NA.shape[1])
    # Loop in every arc, complete from (1) to (-1)
    for i, arc in enumerate(idxs):
        # Node arc:
        NA[arc[0], i] = 1 # From
        NA[arc[1], i] = -1 # To
    arc_idxs = [(arc[0], arc[1]) for arc in idxs]
    return NA, arc_idxs

# Shortest path Utils
def get_selected_arcs(arc_idxs, selected_arcs):
    arc = []
    for idx, i in enumerate(selected_arcs):
        if np.isclose(i, 1, rtol=1e-05, atol=1e-08, equal_nan=False): # Vecinity
            arc.append(arc_idxs[idx])
    return arc

In [2]:
NN = np.array([
        [0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0],])

arcs = np.argwhere(NN)
init_cap=np.array([7, 1, 2, 3, 2, 1, 2]) 
final_cap=np.zeros(init_cap.shape[0],dtype=int)

def dfsearch(NN, initial, goal, path = np.array(0)):
    if initial == goal:
        return True, path
    for newGoal in np.argwhere(NN[:,goal]):
        result, newPath = dfsearch(NN, initial, newGoal.item(0), np.append(path, newGoal) )
        if result:
            return True, newPath
    return False, None

def Index(arcs, s, e):
    for i, arc in enumerate(arcs):
        if arc[0] == s and arc[1] == e:
            return i
    return None

existsPath, path = dfsearch(NN, 0, 5, np.array(5))

while existsPath:
    minimum = np.inf
    toNode = path.item(0)
    for fromNode in path[1:]:
        i = Index(arcs, fromNode, toNode)
        if minimum > init_cap[i]:
            minimum = init_cap[i]
        toNode = fromNode

    toNode = path.item(0)

    for fromNode in path[1:]:
        i = Index(arcs, fromNode, toNode)
        init_cap[i] = init_cap[i] - minimum
        final_cap[i] = final_cap[i] + minimum
        if init_cap[i] == 0: 
            NN[fromNode][toNode] = 0
        toNode = fromNode
    existsPath, path = dfsearch(NN, 0, 5, np.array(5))

print("The flow on each arc is:")
for i in range(arcs.shape[0]):
    print(arcs[i], ": ", final_cap[i])

maxFlow = 0
for index in np.where(init_cap == 0)[0]:
    maxFlow = maxFlow + final_cap[index]

print('\nThe max flow is: {}'.format(maxFlow))

The flow on each arc is:
[0 1] :  4
[0 2] :  1
[1 3] :  1
[1 5] :  3
[2 4] :  1
[3 5] :  1
[4 5] :  1

The max flow is: 5
